### Extracting "Bronx" borough data from Socrata using SQL query

In [1]:
# Import libraries
from pandas.core import api
import datetime
from sodapy import Socrata
import pandas as pd
import folium
import numpy as np
import seaborn as sns

In [2]:
# Initialize the Socrata client
client = Socrata("data.cityofnewyork.us", None)

# Define the current date and one year ago
today = datetime.date.today()
one_year_ago = today - datetime.timedelta(days=365)

# Set the query parameters for Bronx collisions
query_params = (
    f"borough='BRONX' AND "
    f"(number_of_persons_injured>0 OR number_of_persons_killed>0) AND "
    f"crash_date>='{one_year_ago}'"
)

# Query the data for Bronx collisions
results = client.get("h9gi-nx95", where=query_params, limit=10000)

# Create a DataFrame and preprocess it
df = pd.DataFrame.from_records(results)
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# Drop rows with missing latitude or longitude
df = df.dropna(subset=["latitude", "longitude"])

# Define bounding box for Bronx (adjust as necessary)
df = df[(df['latitude'] > 40.7854) & (df['latitude'] < 40.9176)]

# Query ambulance stations in Bronx
query_params = "factype='AMBULANCE STATION' AND boro='BRONX'"
ambulance_stations = client.get("ji82-xba5", where=query_params)

# Convert to DataFrame and preprocess
ambulance_stations_df = pd.DataFrame.from_records(ambulance_stations)
ambulance_stations_df['latitude'] = ambulance_stations_df['latitude'].astype(float)
ambulance_stations_df['longitude'] = ambulance_stations_df['longitude'].astype(float)

# Query hospitals in Bronx
query_params = "factype='HOSPITAL' AND boro='BRONX'"
hospitals = client.get("ji82-xba5", where=query_params)

# Convert to DataFrame and preprocess
hospitals_df = pd.DataFrame.from_records(hospitals)
hospitals_df['latitude'] = hospitals_df['latitude'].astype(float)
hospitals_df['longitude'] = hospitals_df['longitude'].astype(float)


### Viewing dataframe

In [3]:
print(hospitals_df.columns)


Index(['uid', 'facname', 'addressnum', 'streetname', 'address', 'city', 'boro',
       'borocode', 'zipcode', 'latitude', 'longitude', 'xcoord', 'ycoord',
       'bin', 'bbl', 'cd', 'council', 'ct2010', 'ct2020', 'nta2010', 'nta2020',
       'facgroup', 'facsubgrp', 'factype', 'capacity', 'optype', 'opname',
       'opabbrev', 'overlevel', 'overagency', 'overabbrev', 'datasource',
       'facdomain', 'schooldist', 'policeprct', 'servarea', 'geometry'],
      dtype='object')


In [4]:
print(hospitals_df.head(10))

                                uid  \
0  104dc64744405942bc1b491d730a05ff   
1  28a2bf1e3cef8332ec93be1cf2f2474b   
2  3ae3b7ac16d575591c3cb1beaaf9abf5   
3  3f7ab1a0de2506ecd3241bf105c835cc   
4  4b7e554969ec5fe2a40979573494fcd0   
5  534efb625257b404b972a1ec1f1de448   
6  610f6d227ec257b5d54355a96d74fedf   
7  64e3e72fc77d91b81dc5b5653a4c79a1   
8  728f676a523157d6595a8363e027d0fe   
9  e0ac97ef129a1233d60e83f88326c8c1   

                                             facname addressnum  \
0             LINCOLN MEDICAL & MENTAL HEALTH CENTER        234   
1  MONTEFIORE MEDICAL CENTER - HENRY & LUCY MOSES...        111   
2                               CALVARY HOSPITAL INC       1740   
3                                  SBH HEALTH SYSTEM       4422   
4                              JACOBI MEDICAL CENTER       1400   
5  MONTEFIORE MED CENTER - JACK D WEILER HOSP OF ...       1825   
6                          BRONXCARE HOSPITAL CENTER       1276   
7                          BRONXCA

### Identify the nearest hospital for each collision in Bronx

In [5]:
from geopy.distance import geodesic
import folium
import seaborn as sns
import numpy as np

# Function to calculate the nearest hospital for each collision
def find_nearest_facility(collision, facilities):
    collision_location = (collision['latitude'], collision['longitude'])
    distances = facilities.apply(
        lambda x: geodesic(collision_location, (x['latitude'], x['longitude'])).meters, axis=1
    )
    nearest_index = distances.idxmin()
    return facilities.loc[nearest_index, 'facname'], distances[nearest_index]

# Calculate the nearest hospital for each collision
df['nearest_hospital'], df['distance_to_hospital'] = zip(*df.apply(
    lambda x: find_nearest_facility(x, hospitals_df), axis=1
))

# Create a Folium map centered in the Bronx
map_center = [df['latitude'].mean(), df['longitude'].mean()]
collision_map = folium.Map(location=map_center, zoom_start=12)

# Assign unique "Set2" colors for each hospital
set2_colors = sns.color_palette("Set2", len(hospitals_df)).as_hex()
hospital_color_dict = {
    hospitals_df.iloc[i]['facname']: set2_colors[i]
    for i in range(len(hospitals_df))
}

# Add hospital markers with color in the popup
for _, row in hospitals_df.iterrows():
    hospital_name = row['facname']
    hospital_color = hospital_color_dict[hospital_name]  # Get color for the hospital
    folium.Marker(
        [row['latitude'], row['longitude']],
        popup=(
            f"<b>{hospital_name}</b><br>"
            f"Associated Collision Color: <span style='color:{hospital_color}'>{hospital_color}</span>"
        ),
        icon=folium.Icon(color='blue', icon='hospital', prefix='fa')  # Hospital markers remain blue
    ).add_to(collision_map)

# Add collision markers, using "Set2" colors based on nearest hospital
for _, row in df.iterrows():
    nearest_hospital = row['nearest_hospital']
    collision_color = hospital_color_dict[nearest_hospital]  # Assign colorblind color based on nearest hospital
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,  # Adjust the size of the dot
        color=collision_color,  # Color based on nearest hospital
        fill=True,
        fill_color=collision_color,  # Fill color matches outline
        fill_opacity=0.6
    ).add_to(collision_map)

# Save the map to an HTML file
collision_map.save("bronx_collision_map.html")


### Determining the location in Bronx that would minimise response time.

* Haversine Distance Calculation: computes the great-circle distance between two latitude/longitude points.

* Cost Function: calculates the total cost for a given facility location.<br>

    The cost is defined as the sum of the distances from each collision site to its nearest facility, including the new location.

* Using Hill-Climbing Algorithm: explores the neighborhood of a given starting location (can be hospital) to find a local minimum of the cost function.<br>

    The neighborhood is defined by moving north, south, east, and west by a specified step size.


In [6]:
import pandas as pd
from geopy.distance import geodesic
from math import radians, sin, cos, sqrt, atan2
from joblib import Parallel, delayed

# Function to calculate Haversine distance (faster approach than geopy)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radius of Earth in meters
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon2 - lon1)
    a = sin(dphi / 2) ** 2 + cos(phi1) * cos(phi2) * sin(dlambda / 2) ** 2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

# Function to calculate total cost for a given facility location
def calculate_cost(collisions, facilities, new_location):
    total_cost = 0
    new_lat, new_lon = new_location
    for _, collision in collisions.iterrows():
        collision_lat, collision_lon = collision['latitude'], collision['longitude']
        min_distance = float('inf')
        # Calculate distance to all existing and new facilities
        for _, facility in facilities.iterrows():
            facility_lat, facility_lon = facility['latitude'], facility['longitude']
            distance = haversine(collision_lat, collision_lon, facility_lat, facility_lon)
            min_distance = min(min_distance, distance)
        # Include distance to the new location
        distance_to_new = haversine(collision_lat, collision_lon, new_lat, new_lon)
        min_distance = min(min_distance, distance_to_new)
        total_cost += min_distance
    return total_cost

# Function to perform hill-climbing
def hill_climbing(collisions, facilities, starting_point, step_size=1000, max_iterations=50):
    current_location = starting_point
    current_cost = calculate_cost(collisions, facilities, current_location)
    for _ in range(max_iterations):
        neighbors = [
            (current_location[0] + step_size / 111000, current_location[1]),  # North
            (current_location[0] - step_size / 111000, current_location[1]),  # South
            (current_location[0], current_location[1] + step_size / (111000 * cos(radians(current_location[0])))),  # East
            (current_location[0], current_location[1] - step_size / (111000 * cos(radians(current_location[0]))))   # West
        ]
        best_neighbor = None
        best_neighbor_cost = current_cost
        for neighbor in neighbors:
            neighbor_cost = calculate_cost(collisions, facilities, neighbor)
            if neighbor_cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = neighbor_cost
        if best_neighbor_cost < current_cost:
            current_location = best_neighbor
            current_cost = best_neighbor_cost
        else:
            break  # Local optimum reached
    return current_location, current_cost

# Optimization function to find the best location for a new facility
def optimize_new_facility(collisions, facilities):
    best_location = None
    best_cost = float('inf')
    
    results = Parallel(n_jobs=-1)(
        delayed(hill_climbing)(
            collisions, facilities, (facility['latitude'], facility['longitude'])
        ) for _, facility in facilities.iterrows()
    )
    
    for location, cost in results:
        if cost < best_cost:
            best_location = location
            best_cost = cost
    return best_location, best_cost

# Filter collisions and facilities for Bronx
# Assuming df and hospitals_df are pre-loaded DataFrames
bronx_collisions = df[df['borough'] == 'BRONX']
bronx_facilities = hospitals_df[hospitals_df['boro'] == 'BRONX']

# Optimize for Bronx
best_location, best_cost = optimize_new_facility(bronx_collisions, bronx_facilities)

print(f"Best New Location for Bronx: {best_location}")
print(f"Total Cost at Best Location: {best_cost:.2f} meters")


Best New Location for Bronx: (40.822397369691, -73.89131401232832)
Total Cost at Best Location: 5101306.92 meters


### Incorporate best location (new facility) into map

In [7]:
from geopy.distance import geodesic
import folium
import seaborn as sns
import numpy as np
from joblib import Parallel, delayed
from math import radians, sin, cos, sqrt, atan2

# Haversine function for faster distance calculations
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth's radius in meters
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon2 - lon1)
    a = sin(dphi / 2) ** 2 + cos(phi1) * cos(phi2) * sin(dlambda / 2) ** 2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

# Hill-climbing optimization for new facility location
def hill_climbing(collisions, facilities, starting_point, step_size=1000, max_iterations=50):
    current_location = starting_point
    current_cost = calculate_cost(collisions, facilities, current_location)
    for _ in range(max_iterations):
        neighbors = [
            (current_location[0] + step_size / 111000, current_location[1]),  # North
            (current_location[0] - step_size / 111000, current_location[1]),  # South
            (current_location[0], current_location[1] + step_size / (111000 * cos(radians(current_location[0])))),  # East
            (current_location[0], current_location[1] - step_size / (111000 * cos(radians(current_location[0]))))   # West
        ]
        best_neighbor = None
        best_neighbor_cost = current_cost
        for neighbor in neighbors:
            neighbor_cost = calculate_cost(collisions, facilities, neighbor)
            if neighbor_cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = neighbor_cost
        if best_neighbor_cost < current_cost:
            current_location = best_neighbor
            current_cost = best_neighbor_cost
        else:
            break  # Local optimum
    return current_location, current_cost

# Calculate cost (total distances)
def calculate_cost(collisions, facilities, new_location):
    total_cost = 0
    new_lat, new_lon = new_location
    for _, collision in collisions.iterrows():
        collision_lat, collision_lon = collision['latitude'], collision['longitude']
        min_distance = float('inf')
        for _, facility in facilities.iterrows():
            facility_lat, facility_lon = facility['latitude'], facility['longitude']
            min_distance = min(min_distance, haversine(collision_lat, collision_lon, facility_lat, facility_lon))
        total_cost += min(min_distance, haversine(collision_lat, collision_lon, new_lat, new_lon))
    return total_cost

# Optimize location for a new facility
def optimize_new_facility(collisions, facilities):
    results = Parallel(n_jobs=-1)(
        delayed(hill_climbing)(
            collisions, facilities, (facility['latitude'], facility['longitude'])
        ) for _, facility in facilities.iterrows()
    )
    best_location, best_cost = min(results, key=lambda x: x[1])
    return best_location, best_cost

# Nearest hospital function
def find_nearest_facility(collision, facilities):
    collision_location = (collision['latitude'], collision['longitude'])
    distances = facilities.apply(
        lambda x: geodesic(collision_location, (x['latitude'], x['longitude'])).meters, axis=1
    )
    nearest_index = distances.idxmin()
    return facilities.loc[nearest_index, 'facname'], distances[nearest_index]

# Calculate nearest hospital for each collision
df['nearest_hospital'], df['distance_to_hospital'] = zip(*df.apply(
    lambda x: find_nearest_facility(x, hospitals_df), axis=1
))

# Optimize for the best new location
best_location, best_cost = optimize_new_facility(df, hospitals_df)

# Create map
map_center = [df['latitude'].mean(), df['longitude'].mean()]
collision_map = folium.Map(location=map_center, zoom_start=12)

# Assign unique "Set2" colors for each hospital
set2_colors = sns.color_palette("Set2", len(hospitals_df)).as_hex()
hospital_color_dict = {
    hospitals_df.iloc[i]['facname']: set2_colors[i]
    for i in range(len(hospitals_df))
}

# Add hospital markers
for _, row in hospitals_df.iterrows():
    hospital_name = row['facname']
    hospital_color = hospital_color_dict[hospital_name]
    folium.Marker(
        [row['latitude'], row['longitude']],
        popup=(
            f"<b>{hospital_name}</b><br>"
            f"Associated Collision Color: <span style='color:{hospital_color}'>{hospital_color}</span>"
        ),
        icon=folium.Icon(color='blue', icon='hospital', prefix='fa')
    ).add_to(collision_map)

# Add collision markers
for _, row in df.iterrows():
    nearest_hospital = row['nearest_hospital']
    collision_color = hospital_color_dict[nearest_hospital]
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color=collision_color,
        fill=True,
        fill_color=collision_color,
        fill_opacity=0.6
    ).add_to(collision_map)

# Add marker for new facility
folium.Marker(
    location=best_location,
    popup=(
        f"<b>[SUGGESTED] New Facility Location</b><br>"
        f"Latitude: {best_location[0]:.6f}<br>"
        f"Longitude: {best_location[1]:.6f}<br>"
        f"Lowest Cost: {best_cost:.0f} m<br><br>"
        
    ),

    icon=folium.Icon(color='green', icon='plus', prefix='fa')
).add_to(collision_map)

# Save map
collision_map.save("bronx_new_facility_map.html")
